<a href="https://colab.research.google.com/github/Veldora00/Veldora00/blob/main/yfinance_pivot_fib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Fetch Historical Data
def get_forex_data(pair, start, end):
    data = yf.download(pair, start=start, end=end, interval='1d')
    data = data[['Open', 'High', 'Low', 'Close']]
    return data

# Identify Pivot Highs and Lows
def identify_pivots(data, left=20, right=1):
    pivots_high = []
    pivots_low = []

    for i in range(left, len(data) - right):
        is_pivot_high = all(data['High'][i] > data['High'][i - j] for j in range(1, left + 1)) and \
                        all(data['High'][i] > data['High'][i + j] for j in range(1, right + 1))
        is_pivot_low = all(data['Low'][i] < data['Low'][i - j] for j in range(1, left + 1)) and \
                       all(data['Low'][i] < data['Low'][i + j] for j in range(1, right + 1))

        if is_pivot_high:
            pivots_high.append((data.index[i], data['High'][i]))
        if is_pivot_low:
            pivots_low.append((data.index[i], data['Low'][i]))

    return pd.DataFrame(pivots_high, columns=['Date', 'Pivot High']), pd.DataFrame(pivots_low, columns=['Date', 'Pivot Low'])

# Calculate Fibonacci Levels with ±0.2% Tolerance
def calculate_fibonacci_levels(pivot_low, pivot_high):
    levels = {
        '23.6%': pivot_high - 0.236 * (pivot_high - pivot_low),
        '38.2%': pivot_high - 0.382 * (pivot_high - pivot_low),
        '50.0%': pivot_high - 0.500 * (pivot_high - pivot_low),
        '61.8%': pivot_high - 0.618 * (pivot_high - pivot_low),
        '78.6%': pivot_high - 0.786 * (pivot_high - pivot_low),
    }
    return levels

def calculate_fib_tolerance(levels, tolerance_percent=0.002):
    fib_tolerance = {}
    for level_name, level_price in levels.items():
        tolerance_value = level_price * tolerance_percent
        fib_tolerance[level_name] = {
            'Level': level_price,
            'Upper Bound': level_price + tolerance_value,
            'Lower Bound': level_price - tolerance_value
        }
    return fib_tolerance

# Backtest Strategy
def backtest_strategy(data, pivots_high, pivots_low, initial_balance=10000, risk_per_trade=0.005):
    balance = initial_balance
    trades = []

    # Ensure that there are pivot highs and lows
    if pivots_high.empty or pivots_low.empty:
        print("No pivot highs or lows identified.")
        return pd.DataFrame(), balance

    # Identify trend and execute trades based on the trend
    for i in range(1, min(len(pivots_high), len(pivots_low))):
        pivot_high_prev = pivots_high.iloc[i-1]['Pivot High']
        pivot_low_prev = pivots_low.iloc[i-1]['Pivot Low']
        pivot_high = pivots_high.iloc[i]['Pivot High']
        pivot_low = pivots_low.iloc[i]['Pivot Low']

        # Determine trend
        trend = 'neutral'
        if pivot_high > pivot_high_prev and pivot_low > pivot_low_prev:
            trend = 'uptrend'
        elif pivot_low < pivot_low_prev and pivot_high < pivot_high_prev:
            trend = 'downtrend'

        # Calculate Fibonacci levels and tolerance
        fib_levels = calculate_fibonacci_levels(pivot_low_prev, pivot_high_prev)
        fib_tolerance_levels = calculate_fib_tolerance(fib_levels)

        # Check for trade conditions
        for j in range(1, len(data) - 2):
            close_price_day1 = data['Close'].iloc[j]
            open_price_day3 = data['Open'].iloc[j + 2]
            fib_bounce_level = None

            for level_name, bounds in fib_tolerance_levels.items():
                if bounds['Lower Bound'] <= close_price_day1 <= bounds['Upper Bound']:
                    fib_bounce_level = bounds['Level']
                    break

            if fib_bounce_level:
                # Wait for Day 2 candle to close
                close_price_day2 = data['Close'].iloc[j + 1]

                if trend == 'uptrend' and close_price_day2 > fib_bounce_level:
                    # Buy trade on Day 3
                    stop_loss = fib_bounce_level
                    position_size = (balance * risk_per_trade) / abs(open_price_day3 - stop_loss)
                    trades.append({
                        'Date': data.index[j + 2],
                        'Type': 'BUY',
                        'Entry Price': open_price_day3,
                        'Stop Loss': stop_loss,
                        'Position Size': position_size,
                        'Balance Before': balance
                    })
                    balance += position_size * (open_price_day3 - stop_loss)

                elif trend == 'downtrend' and close_price_day2 < fib_bounce_level:
                    # Sell trade on Day 3
                    stop_loss = fib_bounce_level
                    position_size = (balance * risk_per_trade) / abs(open_price_day3 - stop_loss)
                    trades.append({
                        'Date': data.index[j + 2],
                        'Type': 'SELL',
                        'Entry Price': open_price_day3,
                        'Stop Loss': stop_loss,
                        'Position Size': position_size,
                        'Balance Before': balance
                    })
                    balance += position_size * (stop_loss - open_price_day3)

    trades_df = pd.DataFrame(trades)
    trades_df['Balance After'] = trades_df['Balance Before'] + (trades_df['Position Size'] * (trades_df['Entry Price'] - trades_df['Stop Loss']))
    return trades_df, balance

# Analyze Results
def analyze_trades(trades_df, final_balance, initial_balance):
    if trades_df.empty:
        print("No trades executed.")
        return

    total_trades = len(trades_df)
    total_profit = final_balance - initial_balance
    win_rate = trades_df[trades_df['Balance After'] > trades_df['Balance Before']].shape[0] / total_trades if total_trades > 0 else 0

    print(f"Total Trades: {total_trades}")
    print(f"Final Balance: ${final_balance:.2f}")
    print(f"Total Profit: ${total_profit:.2f}")
    print(f"Win Rate: {win_rate:.2%}")

# Save Trades to CSV
def save_trades_to_csv(trades_df, filename='trades.csv'):
    trades_df.to_csv(filename, index=False)
    print(f"Trades have been saved to {filename}")

# Run Backtest
pair = 'EURAUD=X'
start_date = '2020-01-01'
end_date = '2021-01-01'
forex_data = get_forex_data(pair, start_date, end_date)

pivots_high, pivots_low = identify_pivots(forex_data)
trades_df, final_balance = backtest_strategy(forex_data, pivots_high, pivots_low)
analyze_trades(trades_df, final_balance, 10000)

# Save the trades to a CSV file
save_trades_to_csv(trades_df, 'forex_trades_with_confirmation.csv')


[*********************100%***********************]  1 of 1 completed

Total Trades: 207
Final Balance: $16860.98
Total Profit: $6860.98
Win Rate: 73.91%
Trades have been saved to forex_trades_with_confirmation.csv



<ipython-input-13-bcbdb9f28783>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_high = all(data['High'][i] > data['High'][i - j] for j in range(1, left + 1)) and \
<ipython-input-13-bcbdb9f28783>:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_low = all(data['Low'][i] < data['Low'][i - j] for j in range(1, left + 1)) and \
<ipython-input-13-bcbdb9f28783>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  all(da